In [17]:
from __future__ import print_function

import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,TensorBoard,ReduceLROnPlateau

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.activations import relu
from tensorflow.keras.regularizers import l2
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist, cifar10

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from scipy.io import loadmat 
import mat73
from datetime import datetime


print("Packages Loaded")

Packages Loaded


In [2]:
# Load data
# raw_data1 = loadmat('new_echo_cnn_in_out_jstarrs2021_first_try/echo_cnn_in_out_jstars1.mat')
# raw_data1 = loadmat('echo_cnn_in_out_GOOD_layers/new_echo_cnn_in_out_jstars1.mat') 'findpeaks_layers/new_echo_cnn_in_out_jstars1.mat'

raw_data1 = loadmat('../../../Python_Env/final_layers_rowblock15_21/filtered_image/new_echo_cnn_in_out_jstars1.mat')
raw_data2 = loadmat('../../../Python_Env/final_layers_rowblock15_21/filtered_image/new_echo_cnn_in_out_jstars2.mat')
raw_data3 = loadmat('../../../Python_Env/final_layers_rowblock15_21/filtered_image/new_echo_cnn_in_out_jstars3.mat')
# raw_data4 = loadmat('findpeaks_layers_rowblock20/new_echo_cnn_in_out_jstars4.mat')

d1 = raw_data1['echo_cnn1']
t1 = raw_data1['echo_target1']
i1 = raw_data1['echo_idx1']

d2 = raw_data2['echo_cnn2']
t2 = raw_data2['echo_target2']
i2 = raw_data2['echo_idx2']

d3 = raw_data3['echo_cnn3']
t3 = raw_data3['echo_target3']
i3 = raw_data3['echo_idx3']

# d4 = raw_data4['echo_cnn4']
# t4 = raw_data4['echo_target4']
# i4 = raw_data4['echo_idx4']

all_data = np.concatenate( (d1,d2,d3),axis = 0 )
all_target = np.concatenate( (t1,t2,t3),axis = 0 )
all_idx = np.concatenate( (i1,i2,i3),axis = 0 )



In [3]:
# # data_path = '/all_block_data/Dec_Train_block_len_21_011121_2331' # Should start with a forward slash
# # base_path = os.path.join (os.getcwd() + data_path )

# base_path = '..\\all_block_data\Dec_Train_block_len_21_011121_2331'

# # Confirm path is right...
# print(f'{os.path.isdir(base_path)}')

In [4]:
# # raw_data1 = mat73.loadmat('all_block_data/Full_block_len_65_121021_0916/echo_cnn_in_out_jstars_65x15.mat')
# raw_data1 = mat73.loadmat(base_path + '/echo_cnn_in_out_jstars.mat')

# all_data = raw_data1['echo_cnn_input']
# all_target = raw_data1['echo_cnn_target']


In [5]:
row_length = 21
col_length = 15

# Check that the dimension of data is correct
if all_data.shape[1] == row_length*col_length:
    print('Dimensions match')
else:
    print(f' Row block length:{row_length} and col length:{col_length} does not match Data dimension:{all_data.shape[1]}') 
        

Dimensions match


In [6]:
max_class = row_length 

# Highest class is mapped to row_length+1
all_target[all_target == max_class+1 ] = 0


shuffle = 1
if shuffle:
    random.Random(13).shuffle(all_data)
    random.Random(13).shuffle(all_target)
    # random.Random(13).shuffle(all_idx)

## Prep data
train_size = int(np.floor(0.75*len(all_target)));
test_size = int(np.round( 0.2* all_data.shape[0] ))
val_size = all_data.shape[0] -train_size - test_size


x_train = all_data[0:train_size,:]
x_test = all_data[train_size:train_size+test_size,:]
x_val = all_data[-val_size:,:]

y_train = all_target[:train_size]
y_test  = all_target[train_size:train_size+test_size]
y_val = all_target[-val_size:]

var_input_shape = x_train.shape[1] # 240 columns
num_classes = max_class+1 # layers
# Convert labels to categorical orthonormal vectors

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test  = tf.keras.utils.to_categorical(y_test, num_classes)


In [12]:
# Create your own ResNet here.

shp = x_train.shape[1]

# Skip 3 connection version of base model
drop_out = 0.3
inputs = tf.keras.Input(shape=(shp,), name='img')
x = Dense(512, activation='relu')(inputs)
block_1_output = Dense(512, activation='relu')(x)

x = Dense(512)(block_1_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)

x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)

x = Dense(512)(x)
x = BatchNormalization()(x)

block_2_output = tf.keras.layers.add([x, block_1_output])

x = Dense(512)(block_2_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_3_output = tf.keras.layers.add([x, block_2_output])

x = Dense(512)(block_3_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_4_output = tf.keras.layers.add([x, block_3_output])

x = Dense(512)(block_4_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_5_output = tf.keras.layers.add([x, block_4_output])


x = Dense(512)(block_5_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_6_output = tf.keras.layers.add([x, block_5_output])

x = Dense(512)(block_6_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_7_output = tf.keras.layers.add([x, block_6_output])

x = Dense(512)(block_7_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_8_output = tf.keras.layers.add([x, block_7_output])

x = Dense(512)(block_8_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_9_output = tf.keras.layers.add([x, block_8_output])

x = Dense(512)(block_9_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_10_output = tf.keras.layers.add([x, block_9_output])


x = Dense(512, activation='relu')(block_10_output)
x = Dropout(drop_out)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs, name='resnet')

time_stamp = datetime.strftime( datetime.now(),'%d_%B_%y_%H%M')

model.compile( Adam(amsgrad=True),loss= 'categorical_crossentropy', metrics=['accuracy'])  #tf.keras.losses.KLDivergence()
# earlyStopping = EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='min')

# checkpoint_filepath = f'{base_path}_/tmp/{time_stamp}_jstars_Epoch_{epoch:02d}_ValLoss{val_loss:.2f}.hdf5'
# checkpoint_callback = ModelCheckpoint(checkpoint_filepath, save_best_only=True, monitor='val_accuracy', mode='min')
# reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=0, emin_delta =1e-4, mode='min')


model.fit(x_train, y_train,
          epochs= 10, 
          batch_size= 64, 
          validation_data=(x_test, y_test)) #,reduce_lr_loss , callbacks=[checkpoint_callback]


Epoch 1/10
24544/24544 [==============================] - 3495s 142ms/step - loss: 1.1756 - accuracy: 0.5526 - val_loss: 1.7957 - val_accuracy: 0.4350
Epoch 2/10
24544/24544 [==============================] - 9197s 375ms/step - loss: 1.0780 - accuracy: 0.5831 - val_loss: 2.0002 - val_accuracy: 0.3611
Epoch 3/10
24544/24544 [==============================] - 3435s 140ms/step - loss: 1.0482 - accuracy: 0.5926 - val_loss: 1.3920 - val_accuracy: 0.4857
Epoch 4/10
24544/24544 [==============================] - 3436s 140ms/step - loss: 1.0283 - accuracy: 0.5998 - val_loss: 1.2410 - val_accuracy: 0.5578
Epoch 5/10
24544/24544 [==============================] - 3427s 140ms/step - loss: 1.0137 - accuracy: 0.6048 - val_loss: 1.4722 - val_accuracy: 0.4972
Epoch 6/10
24544/24544 [==============================] - 3465s 141ms/step - loss: 1.0014 - accuracy: 0.6093 - val_loss: 1.3145 - val_accuracy: 0.5301
Epoch 7/10
24544/24544 [==============================] - 3400s 139ms/step - loss: 0.9911 - ac

In [13]:

_, accuracy = model.evaluate(x_test, y_test) #, top_5_accuracy
print(f"Test accuracy: {round(accuracy * 100, 2)}%")
# print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")


model.save(f'{base_path}/CopyofResNetCNN_weights/{ipynbname.name()}_{time_stamp}_Acc_{accuracy: .3f}_{row_length}x{col_length}.h5')

13090/13090 [==============================] - 393s 30ms/step - loss: 1.3691 - accuracy: 0.5262
Test accuracy: 52.62%


NameError: name '__file__' is not defined

In [26]:
# Train further
train_further = 1
if train_further:
    model.fit(x_train, y_train,
          epochs= 50, 
          batch_size= 64, 
          validation_data=(x_test, y_test)) #mcp_save,

    model.save(f'{base_path}/CopyofResNetCNN_weights/{ipynbname.name()}_{time_stamp}_trainfurther_Acc_{accuracy: .3f}_{row_length}x{col_length}.h5')

Epoch 1/50
24544/24544 [==============================] - 3218s 131ms/step - loss: 0.9566 - accuracy: 0.6250 - val_loss: 1.3808 - val_accuracy: 0.5353
Epoch 2/50
24544/24544 [==============================] - 1179s 48ms/step - loss: 0.9501 - accuracy: 0.6267 - val_loss: 1.4687 - val_accuracy: 0.4893
Epoch 3/50
24544/24544 [==============================] - 1497s 61ms/step - loss: 0.9423 - accuracy: 0.6291 - val_loss: 1.1906 - val_accuracy: 0.5590
Epoch 4/50
24544/24544 [==============================] - 2077s 85ms/step - loss: 0.9336 - accuracy: 0.6324 - val_loss: 1.1486 - val_accuracy: 0.5816
Epoch 5/50
24544/24544 [==============================] - 1646s 67ms/step - loss: 0.9243 - accuracy: 0.6347 - val_loss: 1.4107 - val_accuracy: 0.4917
Epoch 6/50
24544/24544 [==============================] - 1965s 80ms/step - loss: 0.9157 - accuracy: 0.6379 - val_loss: 1.1269 - val_accuracy: 0.5917
Epoch 7/50
24544/24544 [==============================] - 1152s 47ms/step - loss: 0.9053 - accuracy

In [19]:
# Import Test Set

test_data = mat73.loadmat('Y:\\ibikunle\\Python_Project\\Fall_2021\\all_block_data\Dec_Test_block_len_21_021121_0948\echo_cnn_in_out_jstars.mat')

loaded_x_test = test_data['echo_cnn_input']
loaded_y_test = test_data['echo_cnn_target']

# loaded_x_test = np.reshape( loaded_x_test, (loaded_x_test.shape[0],max_class,-1) )
# loaded_x_test = np.transpose(loaded_x_test,(0,2,1))

# Highest class is mapped to row_length+1
loaded_y_test[loaded_y_test == max_class+1 ] = 0

In [ ]:
# Test on training data

y_pred_test = [np.argmax(model.predict(np.expand_dims(loaded_x_test[idx],axis=0))) for idx in range(len(loaded_y_test)) ]

combined_result = zip(y_val,y_pred)

exact_accuracy = 100 * sum([1 if y_pred_test[idx]==loaded_y_test[idx]   else 0 for idx in range(len(y_pred_test)) ]) / len(y_pred_test)
test_5_pixel_margin_accuracy = 100 * sum([1 if abs(y_pred_test[idx]-loaded_y_test[idx]) < 5 else 0 for idx in range(len(y_pred_test)) ]) / len(y_pred_test)

print(f' Exact accuracy is {exact_accuracy: .5f} %')
print(f' Top 5% accuracy is {test_5_pixel_margin_accuracy: .5f} %')

In [ ]:
loaded_model = tf.keras.models.load_model('../../../Python_Env/jstars_weight_21_15_july2021_correct.h5')

In [ ]:
# Predict raw_new_data

y_pred_loaded = [np.argmax(loaded_model.predict(np.reshape(x_val[idx,:],(1,var_input_shape)))) for idx in range(len(x_val)) ]
# y_pred = [(int(all_target[idx]),np.argmax(model.predict(np.reshape(all_data[idx,:],(1,var_input_shape)))) ) for idx in range(1001000,1002000) ]
test_accuracy_margin = 100 * sum([1 if abs(y_pred[idx]-y_val[idx]) < 5 else 0 for idx in range(len(y_pred)) ]) / len(y_pred)

print(f'Accuracy is {test_accuracy_margin} %') # Previous Accuracy is 86.99837239583333 %

In [ ]:
model_predictions = np.asarray(y_pred)
ground_truth = y_val.squeeze()

In [ ]:
# (A) Case where prediction was zero
zero_predictions_vs_gt = np.asarray( list(zip( model_predictions[model_predictions == 0], ground_truth[model_predictions == 0])) )
# (A) (ii)Case where prediction was zero but ground truth is different
zero_predictions_wrong = ( sum( [1 if iter[0] !=iter[1] else 0 for iter in zero_predictions_vs_gt ] ) ,len(zero_predictions_vs_gt) )
   
#(B) Case where ground truth was zero 
zero_gt_vs_predictions =  np.asarray( list(zip( ground_truth[ground_truth == 0], model_predictions[ground_truth == 0] )) ) 
#(B) (i) Case where ground truth was zero but prediction was different
zero_gt_right_pred= ( sum( [1 if iter[0] ==iter[1] else 0 for iter in zero_gt_vs_predictions ] ) ,len(zero_gt_vs_predictions) )



In [ ]:
# Prediction per Ground-truth class
class_predictions = [ model_predictions[ground_truth==idx]  for idx in range(num_classes) ]

# RMSE without the zero class for each class
# named_class_rmse = [('Class-%s'%(each_class),np.sqrt (np.mean((each_class - class_predictions[each_class])**2)) ) for each_class in range(1,17) ]
# per_class_rmse = np.asarray( [ np.sqrt (np.mean((each_class - class_predictions[each_class])**2)) for each_class in range(1,17)]  )  


In [ ]:
# Weight of each class
(_,cl_count) = np.unique(ground_truth,return_counts = True)
weight = cl_count/len(ground_truth)

total_rmse = np.mean( per_class_rmse* weight[1:] )



In [ ]:
# Histogram of each class prediction
for idx,each_class in enumerate(class_predictions):
    plt.figure()
    plt.title('Class %s'%idx)
    _ = plt.hist(each_class, bins=np.arange(-0.5,22.5,1));
    

In [ ]:
def PlotCurves(model):
    
    plt.plot(model.history.history['accuracy'],label = 'train')
    plt.grid(color = 'r', linestyle = '--', linewidth = .1)
    plt.ylabel('Accuracy');
    plt.title('Training accuracy');
    plt.figure()
    plt.plot(model.history.history['val_accuracy'],label = 'validation')
    plt.grid(color = 'r', linestyle = '--', linewidth = .1)
    plt.ylabel('Val Accuracy');
    plt.title('Validation accuracy');

    return None

PlotCurves(model)

In [ ]:
# Evaluate on test set ( this should be evaluation set actually)
y_test1  = all_target[train_size:]
predictions = [(np.argmax(loaded_model.predict(np.reshape(x_test[idx,:],(1,240)))),int(y_test1[idx])) for idx in range(len(x_test)) ]

# Confusion matrix
confusion_mtx = np.zeros((17,17) )

for pred,actual in predictions:
    confusion_mtx[pred,actual] +=1

In [ ]:
time_stamp = datetime.strftime( datetime.now(),'%d_%B_%y_%H%M')

model.save(f'FullResNetModel_july2021_w21_15_{time_stamp}.h5')